In [ ]:
%load_ext autoreload
%load_ext line_profiler

In [ ]:
import os, sys
import holoviews as hv
sys.path.append(os.path.abspath('../../two_dim_majoranas/'))

import inspect
from IPython import display

import adaptive
adaptive.notebook_extension()

import numpy as np
import scipy.constants
import cmath
import kwant 

import functools as ft

import sns_system, plotting_results
import topology, spectrum, supercurrent
from distributed_sns import AggregatesSimulationSet as ASS
from distributed_sns import SimulationSet as SS

# Define system

In [ ]:
constants = dict(
    m_eff=0.023 * scipy.constants.m_e / (scipy.constants.eV * 1e-3) / 1e18,  # effective mass in kg, 
    hbar=scipy.constants.hbar / (scipy.constants.eV * 1e-3),
    e = scipy.constants.e,
    current_unit=scipy.constants.k * scipy.constants.e / scipy.constants.hbar * 1e9,  # to get nA
    mu_B=scipy.constants.physical_constants['Bohr magneton'][0] / (scipy.constants.eV * 1e-3),
    k=scipy.constants.k / (scipy.constants.eV * 1e-3),
    exp=cmath.exp,
    cos=cmath.cos,
    sin=cmath.sin
   )

syst_pars = {'Ll' :  1000,
             'Lr' : 1000,
             'Lm' : 250,
             'Ly' : 25,
             'a' :  25,
             'mu_from_bottom_of_spin_orbit_bands': True}

lat_spacings = [5, 12.5, 15.625, 25]
syst_pars_dict = {a : syst_pars.copy() for a in lat_spacings}
[v.update(a=k) for k,v in syst_pars_dict.items()]

params_raw= dict(g_factor_middle = 10,
                 g_factor_left = 0,
                 g_factor_right = 0,
                 mu = 10.0,
                 alpha_middle = 28,
                 alpha_left = 28,
                 alpha_right = 28,
                 Delta_left = 0.18,
                 Delta_right = 0.18,
                 B = 0.5,
                 phase = np.pi/2,
                 T = .25,
                 V = 0)

params = dict(**constants,
              **params_raw)

# Make SNS junction

In [ ]:
syst_dict = {k: sns_system.make_wrapped_system(**v) for k,v in syst_pars_dict.items()}

# Get current source code

In [ ]:
wrapped_current_implementation = inspect.getsource(supercurrent.wrapped_current)
display.Code(data=wrapped_current_implementation, language='python')
exec(inspect.getsource(supercurrent))

# Make local version

In [ ]:
a = 12.5

In [ ]:
syst_pars = syst_pars_dict[a]
tol=1e-2
syst_wrapped=syst_dict[a]

cut_tag=0
direction=0

(cut_indices, cut_sites) = get_cut_sites_and_indices(syst_wrapped, cut_tag, direction)
current_operator = kwant.operator.Current(syst_wrapped,
                                          onsite=sigz,
                                          where=cut_sites)



def f(k):
    p = dict(k_y=k, **params)
    ham = syst_wrapped.hamiltonian_submatrix(params=p)
    local_current_operator = current_operator.bind(params=p)
    
    (en, evs) = np.linalg.eigh(ham)
    I = sum(fermi_dirac(e.real, p) * local_current_operator(ev)
        for (e, ev) in zip(en, evs.T))
    return sum(I)*params['e']/params['hbar']/a


In [ ]:
ef_max = params['mu'] + params['m_eff']*params['alpha_middle']**2/2/params['hbar']**2
kf = np.sqrt(ef_max*2*params['m_eff'])/params['hbar']*syst_pars['a']
kmax = min(1.5*kf, np.pi)
learner = adaptive.IntegratorLearner(f, [0, kmax], tol)

In [ ]:
runner = adaptive.Runner(learner, lambda l: l.done())
runner.live_info()
# runner = adaptive.runner.simple(learner, lambda l: l.done())

In [ ]:
# 12.5 
learner.igral

In [ ]:
# 15.625 -2.8210614633668415e-09
learner.igral

In [ ]:
learner.plot()

In [ ]:
learner.plot()

In [ ]:
hv.Points(learner.done_points)


In [ ]:
runner.overhead()

In [ ]:
%lprun -f f f(0.1)

In [ ]:
syst_wrapped = syst_dict[12.5]
p = dict(k_y=.1, **params)
ham = syst_wrapped.hamiltonian_submatrix(params=p)
local_current_operator = current_operator.bind(params=p)

(en, evs) = np.linalg.eigh(ham)

I = sum(local_current_operator(ev) for (e, ev) in zip(en, evs.T))

In [ ]:
I

In [ ]:
params['mu'] = .5/params['m_eff']*params['hbar']**2*kf**2

In [ ]:
np.sqrt(params['mu']*2*params['m_eff'])/params['hbar']*12.5

In [ ]:
J*kg
J^2 s^2

kg^2 m^2 s^-2
kg^2 m^4 s^-4 s^2

m^-2

